In [2]:
import sys
import unittest
import os
import datetime

sys.path.append('/Users/jacobrosen/Desktop/UBC/Courses/533 - Collaborative Software Dev/Data-533-Group-16-project/mybudgeter')
from database.budget import Budget
from database.transactions import Transactions

In [4]:
class TestBudget (unittest.TestCase):
    @classmethod
    def setUpClass(self):
        # sets up a new db in cwd 
        self.budget = Budget()

    @classmethod
    def tearDownClass(self):
        # disconnect from db and remove it from directory 
        del self.budget
        os.remove(os.path.join(os.getcwd(), 'budgeting.db'))

    def setUp(self):
        # initialize table with two values in it to test with
        self.budget.add_from_lists(['food', 'clothing'], [300, 100])
        self.date = datetime.datetime.now()

    def tearDown(self):
        # clear the table to start fresh
        self.budget.query('delete from budget')

    # test add_category
    def test_add_category(self):
        self.budget.add_category('school')
        self.assertIn(('school', self.date.month, self.date.year, 0), self.budget.check_budget())
        self.budget.add_category('health', 150)
        self.assertIn(('health', self.date.month, self.date.year, 150), self.budget.check_budget())
        last_month = self.date.replace(month = self.date.month-1)
        self.budget.add_category('health', 100, last_month)
        self.assertIn(('health', last_month.month, last_month.year, 100), self.budget.check_budget(last_month.month, last_month.year))

    # test modify category
    def test_modify(self):
        self.budget.modify_category('food', 120, self.date.month, self.date.year)
        self.assertIn(('food', self.date.month, self.date.year, 120), self.budget.check_budget())
        self.budget.modify_category('clothing', 300, self.date.month, self.date.year)
        self.assertIn(('clothing', self.date.month, self.date.year, 300), self.budget.check_budget())

    # test delete
    def test_delete(self):
        self.budget.delete_category('food', self.date.month, self.date.year)
        self.assertNotIn(('food', self.date.month, self.date.year, 300), self.budget.check_budget())
        self.budget.delete_category('clothing', self.date.month, self.date.year)
        self.assertNotIn(('clothing', self.date.month, self.date.year, 100), self.budget.check_budget())

    # test check
    def test_check(self):
        self.assertEqual(self.budget.check_budget(), [('food', self.date.month, self.date.year, 300), ('clothing', self.date.month, self.date.year, 100)])
        self.assertEqual(self.budget.check_budget(year=1999), [])
        self.budget.add_category('test', 100, datetime.datetime(1999, 5, 17))
        self.assertEqual(self.budget.check_budget(5, 1999), [('test', 5, 1999, 100)])
        
    # test get
    def test_get_categories(self):
        self.assertEqual(self.budget.get_categories(), ['clothing', 'food'])
        self.budget.add_category('test')
        self.assertEqual(self.budget.get_categories(), ['clothing', 'food', 'test'])
        self.budget.modify_category('test', 10, self.date.month, self.date.year)
        self.assertEqual(self.budget.get_categories(), ['clothing', 'food', 'test'])
        self.budget.delete_category('food', self.date.month, self.date.year)
        self.assertEqual(self.budget.get_categories(), ['clothing', 'test'])

unittest.main(argv=[''], verbosity=2, exit=False)

test_add_category (__main__.TestBudget) ... 

ok
test_check (__main__.TestBudget) ... ok
test_delete (__main__.TestBudget) ... ok
test_get_categories (__main__.TestBudget) ... ok
test_modify (__main__.TestBudget) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.102s

OK


In [22]:
from budget_test import TestBudget

In [5]:
class TestTransactions(unittest.TestCase):
    @classmethod
    def setUpClass(self):
        # sets up a new db in cwd 
        self.trans = Transactions()

    @classmethod
    def tearDownClass(self):
        # disconnect from db and remove it from directory 
        del self.trans
        os.remove(os.path.join(os.getcwd(), 'transactions.db'))

    def setUp(self):
        # initialize table with transactions in it to test with
        self.dec_1 = datetime.datetime(2023, 12, 1)
        t1 = self.trans.add_transaction('school', 15, name='books')
        t2 = self.trans.add_transaction('food',10, date=self.dec_1, name='lunch')
        t3 = self.trans.add_transaction('food', 20, date=self.dec_1, name='dinner')
        t4 = self.trans.add_transaction('clothing', 30, name='shirt')
        self.date = datetime.datetime.now()
        self.trans_list = [t1, t2, t3, t4]

    def tearDown(self):
        # clear the table to start fresh
        self.trans.query('delete from transactions')
    
    def test_add_transaction(self):
        id1 = self.trans.add_transaction('food', 30)
        self.assertIn((id1, self.date.strftime("%Y-%m-%d"), None, 'food', 30), self.trans.get_n_transactions(5))
        id2 = self.trans.add_transaction('school', 5, self.dec_1)
        self.assertIn((id2, self.dec_1.strftime("%Y-%m-%d"), None, 'school', 5), self.trans.get_n_transactions(6))
        id3 = self.trans.add_transaction('test', 5, self.dec_1, 'testing a name')
        self.assertIn((id3, self.dec_1.strftime("%Y-%m-%d"), 'testing a name', 'test', 5), self.trans.get_n_transactions(7))

    def test_modify_transaction(self):
        self.trans.modify_transaction(self.trans_list[0], 'amount', 100)
        self.assertIn((self.trans_list[0], self.date.strftime("%Y-%m-%d"), 'books', 'school', 100), self.trans.get_n_transactions(4))
        self.trans.modify_transaction(self.trans_list[3], 'trans_date', '2000-12-2')
        self.assertIn((self.trans_list[3], '2000-12-2', 'shirt', 'clothing', 30), self.trans.get_n_transactions(4))
        self.trans.modify_transaction(self.trans_list[1], 'category', 'shopping')
        self.assertIn((self.trans_list[1], self.dec_1.strftime("%Y-%m-%d"), 'lunch', 'shopping', 10), self.trans.get_n_transactions(4))
        self.trans.modify_transaction(self.trans_list[2], 'name', 'breakfast')
        self.assertIn((self.trans_list[2], self.dec_1.strftime("%Y-%m-%d"), 'breakfast', 'food', 20), self.trans.get_n_transactions(4))

    def test_delete_transaction(self):
        self.trans.delete_transaction(self.trans_list[0])
        self.assertNotIn((self.trans_list[0], self.date.strftime("%Y-%m-%d"), 'books', 'school', 15), self.trans.get_n_transactions(4))
        self.assertEqual(len(self.trans.get_n_transactions(4)), 3)
        self.trans.delete_transaction(self.trans_list[3])
        self.assertNotIn((self.trans_list[3], self.date.strftime("%Y-%m-%d"), 'shirt', 'clothing', 30), self.trans.get_n_transactions(4))
        self.assertEqual(len(self.trans.get_n_transactions(4)), 2)

    def test_get_n_transactions(self):
        self.assertEqual(self.trans.get_n_transactions(1), [(self.trans_list[0], self.date.strftime("%Y-%m-%d"), 'books', 'school', 15)])
        self.assertEqual(self.trans.get_n_transactions(1, asc=True), [(self.trans_list[1], self.dec_1.strftime("%Y-%m-%d"), 'lunch', 'food', 10)])
        list1 = [(self.trans_list[3], self.date.strftime("%Y-%m-%d"), 'shirt', 'clothing', 30), (self.trans_list[2], self.dec_1.strftime("%Y-%m-%d"), 'dinner', 'food', 20)]
        self.assertEqual(self.trans.get_n_transactions(2, sort_field="amount", asc=False), list1)

unittest.main(argv=[''], verbosity=2, exit=False)

test_add_category (__main__.TestBudget) ... ok
test_check (__main__.TestBudget) ... ok
test_delete (__main__.TestBudget) ... ok
test_get_categories (__main__.TestBudget) ... ok
test_modify (__main__.TestBudget) ... ok
test_add_transaction (__main__.TestTransactions) ... ok
test_delete_transaction (__main__.TestTransactions) ... ok
test_get_n_transactions (__main__.TestTransactions) ... ok
test_modify_transaction (__main__.TestTransactions) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.206s

OK


In [7]:
def my_suite(): 
	suite = unittest.TestSuite()
	result = unittest.TestResult()
	# add tests for budget
	# suite.addTest(TestBudget('test_add_category'))
	# suite.addTest(TestBudget('test_modify'))
	# suite.addTest(TestBudget('test_delete'))
	# suite.addTest(TestBudget('test_check'))
	# suite.addTest(TestBudget('test_get_categories'))
	suite.addTest(unittest.makeSuite(TestBudget))

	# add tests for transactions
	# suite.addTest(TestTransactions('test_add_transaction'))
	# suite.addTest(TestTransactions('test_modify_transaction'))
	# suite.addTest(TestTransactions('test_delete_transaction'))
	# suite.addTest(TestTransactions('test_get_n_transactions'))
	suite.addTest(unittest.makeSuite(TestTransactions))

	runner = unittest.TextTestRunner() 
	print(runner.run(suite))

my_suite() 

.........
----------------------------------------------------------------------
Ran 9 tests in 0.098s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>
